# Notebook Lecture 5: Transfer Functions
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook covers the basics of response to exponential inputs, transfer functions, conversions between state-space models and transfer functions, and connections to Laplace transforms. 

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives

In previous lectures, we've seen that this course addresses the control of dynamical systems. Two key course objectives are to understand how to analyze and synthesize systems to achieve specific goals. To do this effectively, we need to understand various ways of representing and analyzing dynamical systems mathematically.
In this notebook, we delve deeper into some fundamental concepts and techniques used in control systems analysis. We'll explore system responses and different system representations.
After completing this notebook, you should be able to:

- Understand and analyze a system's response to exponential inputs.
- Interpret and work with transfer functions.
- Convert between state-space models and transfer functions, and understand the benefits of each representation.
- Appreciate the role of Laplace transforms in control systems analysis and how they relate to system behavior.
